<a href="https://colab.research.google.com/github/baochi0212/facemask/blob/master/My_YOLOV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The prev facemask use the retina net(pyramid)

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10878, done.
remote: Total 10878 (delta 0), reused 0 (delta 0), pack-reused 10878
Receiving objects: 100% (10878/10878), 10.96 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (7512/7512), done.


In [ ]:
!pip install -r /content/yolov5/requirements.txt

In [1]:
!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection
!unzip -q Face\ Mask\ Detection.zip

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 9), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.
Face Mask Detection.zip is done!


In [2]:
import os
import random
import numpy as np
import shutil

print(len(os.listdir('annotations')))
print(len(os.listdir('images')))

!mkdir test_images
!mkdir test_annotations


random.seed(1234)
idx = random.sample(range(853), 170)

for img in np.array(sorted(os.listdir('images')))[idx]:
    shutil.move('images/'+img, 'test_images/'+img)

for annot in np.array(sorted(os.listdir('annotations')))[idx]:
    shutil.move('annotations/'+annot, 'test_annotations/'+annot)

print(len(os.listdir('annotations')))
print(len(os.listdir('images')))
print(len(os.listdir('test_annotations')))
print(len(os.listdir('test_images')))

853
853
683
683
170
170


In [37]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from bs4 import BeautifulSoup
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
from matplotlib import pyplot as plt
import os

def generate_box(obj):
    
    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)
    
    return [xmin, ymin, xmax, ymax]

def generate_label(obj):

    if obj.find('name').text == "with_mask":

        return 1

    elif obj.find('name').text == "mask_weared_incorrect":

        return 2

    return 0

def generate_target(file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        objects = soup.find_all("object")

        num_objs = len(objects)

        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))

        boxes = torch.as_tensor(boxes, dtype=torch.float32) 
        labels = torch.as_tensor(labels, dtype=torch.int64) 
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        return target

def plot_image_from_output(img, annotation):
    
    img = img.cpu().permute(1,2,0)
    
    rects = []

    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 0 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        
        elif annotation['labels'][idx] == 1 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        else :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')

        rects.append(rect)

    return img, rects

class MaskDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)
        
        if 'test' in self.path:
            label_path = os.path.join("test_annotations/", file_label)
        else:
            label_path = os.path.join("annotations/", file_label)

        img = Image.open(img_path).convert("RGB")
        target = generate_target(label_path)
        
        to_tensor = torchvision.transforms.ToTensor()

        if self.transform:
            img, transform_target = self.transform(np.array(img), np.array(target['boxes']))
            target['boxes'] = torch.as_tensor(transform_target)

        # change to tensor
        img = to_tensor(img)


        return img, target

def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset('images/')
test_dataset = MaskDataset('test_images/')

data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=collate_fn)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, collate_fn=collate_fn)


In [36]:
a = [[1,2,3], [4,5,6]]
print(list(zip(*a)))
#* take a as combination of list zip(a, b) not zip(a)

[(1, 4), (2, 5), (3, 6)]


In [38]:
iter(data_loader).next()[1][0], iter(data_loader).next()[0][0].shape
#format: data - label(boxes, labels(binary))
#different size so we have to pass the custom collate_fn

({'boxes': tensor([[ 79., 105., 109., 142.],
          [185., 100., 226., 144.],
          [325.,  90., 360., 141.]]), 'labels': tensor([0, 1, 0])},
 torch.Size([3, 366, 512]))

In [25]:
dataset[1][0].shape, dataset[0][0]

(torch.Size([3, 156, 400]),
 tensor([[[0.4431, 0.4314, 0.3843,  ..., 0.1569, 0.2824, 0.3373],
          [0.4118, 0.3882, 0.3451,  ..., 0.1529, 0.1882, 0.3098],
          [0.3765, 0.3569, 0.3255,  ..., 0.1490, 0.1216, 0.2784],
          ...,
          [0.3333, 0.3412, 0.3569,  ..., 0.1098, 0.1098, 0.1137],
          [0.3373, 0.3490, 0.3608,  ..., 0.1137, 0.1137, 0.1176],
          [0.3647, 0.3765, 0.3922,  ..., 0.1176, 0.1176, 0.1216]],
 
         [[0.4353, 0.4235, 0.3765,  ..., 0.2314, 0.3843, 0.4549],
          [0.3922, 0.3804, 0.3373,  ..., 0.2275, 0.2863, 0.4275],
          [0.3608, 0.3412, 0.3098,  ..., 0.2235, 0.2235, 0.3922],
          ...,
          [0.3255, 0.3333, 0.3490,  ..., 0.1059, 0.1059, 0.1098],
          [0.3294, 0.3412, 0.3529,  ..., 0.1098, 0.1098, 0.1137],
          [0.3569, 0.3686, 0.3843,  ..., 0.1137, 0.1137, 0.1176]],
 
         [[0.4549, 0.4431, 0.3961,  ..., 0.4000, 0.6157, 0.7294],
          [0.4157, 0.3922, 0.3490,  ..., 0.3843, 0.5059, 0.6784],
          [0

Use roboflow to convert to yolov5 coco


In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QlPOQtcxpcmV25dVhFGb")
project = rf.workspace().project("face_mask-oxref")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to face_mask-1 in yolov5pytorch:: 100%|██████████| 1708/1708 [00:01<00:00, 1099.99it/s]


In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10882, done.
remote: Total 10882 (delta 0), reused 0 (delta 0), pack-reused 10882
Receiving objects: 100% (10882/10882), 10.92 MiB | 26.94 MiB/s, done.
Resolving deltas: 100% (7524/7524), done.


In [9]:
%cd yolov5

/content/yolov5


In [ ]:
#run yolo
!python train.py --img 416 --batch 16 --epochs 150 --data /content/yolov5/face_mask-1/data.yaml --weights /content/yolov5s.pt --cache

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source /content/yolov5/face_mask-1/test/images

In [11]:
from glob import glob
from IPython.display import Image, display

for image in glob('/content/yolov5/runs/detect/exp/*.jpg'):
  display(Image(image))
  print()
  


['/content/yolov5/face_mask-1/test/images/maksssksksss2_png.rf.1b54188e1ac11bc0c0b6cd02bc7a0d0d.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss48_png.rf.3c9c7edc119b02416025394f77598b4a.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss664_png.rf.6b5866f9176d68ee1a01438c2ce99e2f.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss844_png.rf.34d5979504863e738582be2fa1a10594.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss840_png.rf.c13a9c9de6fe0dfcda206c81c94001c8.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss197_png.rf.2be6db9a470681e6873422f4efc3fe73.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss278_png.rf.054e88571a82505f649e10ba7e94edbb.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss265_png.rf.e812dfb43b3e980b58782b6508b1cb0e.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss670_png.rf.dae672cd1aa53a15e0fde60dce675f23.jpg',
 '/content/yolov5/face_mask-1/test/images/maksssksksss451_png.rf.47